**1. Import Dependencies**

In [1]:
import os
import numpy as np
import pandas as pd
import time

In [2]:
import warnings

from typing import Any, Callable, Dict, List, Optional, Tuple, Union

import gym
from stable_baselines3 import PPO      #PPO -> Proximal Policy Optimization
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy  #to evaluate the model 
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common import type_aliases
from stable_baselines3.common.vec_env import DummyVecEnv, VecEnv, VecMonitor, is_vecenv_wrapped

**2.Train Environment**

In [3]:
# Default environment
# from gym.wrappers.time_limit import TimeLimit
# from car_racing_default import CarRacingDefault
# env = TimeLimit(CarRacingDefault(),max_episode_steps=1000)
# eval_env= env

# Curriculum environment
# from gym.wrappers.time_limit import TimeLimit
# from TurnRates.car_racing_curriculum import CarRacingCurriculum
# env = TimeLimit(CarRacingCurriculum(),max_episode_steps=1000)

# Evaluation environment
# from gym.wrappers.time_limit import TimeLimit
# from TurnRates.car_racing_eval import CarRacingEval
# eval_env = TimeLimit(CarRacingEval(),max_episode_steps=1000)

# Default obstacles environment
# from gym.wrappers.time_limit import TimeLimit
# from car_racing_obstacles import CarRacingObstacles
# env = TimeLimit(CarRacingObstacles(),max_episode_steps=1000)
# eval_env= env

# Curriculum obstacles environment
# from gym.wrappers.time_limit import TimeLimit
# from Obstacles.car_racing_obstacles_curriculum import CarRacingObstaclesCurriculum
# env = TimeLimit(CarRacingObstaclesCurriculum(),max_episode_steps=1000)

# Obstacles Evaluation environment
from gym.wrappers.time_limit import TimeLimit
from Obstacles.car_racing_obstacles_eval import CarRacingObstaclesEval
eval_env = TimeLimit(CarRacingObstaclesEval(),max_episode_steps=1000)

# Curriculum both environment
# from gym.wrappers.time_limit import TimeLimit
# from Both.car_racing_obstacles_curriculum_both import CarRacingObstaclesCurriculumBoth
# env = TimeLimit(CarRacingObstaclesCurriculumBoth(),max_episode_steps=1000)

# Evaluation both environment
# from gym.wrappers.time_limit import TimeLimit
# from Both.car_racing_obstacles_eval_both import CarRacingObstaclesEvalBoth
# eval_env = TimeLimit(CarRacingObstaclesEvalBoth(),max_episode_steps=1000)


**3.Training Model**

In [4]:
seed=0

In [5]:
log_path = os.path.join('./runs')

# Learning Rates: 
# 0.0005- PPO Default, 0.00025- PPO Curriculum
# 0.0005- PPO Obstacles, 0.000475- PPO Obstacles Curriculum
# 0.0002- PPO Both Curriculum

model = PPO('CnnPolicy', env, learning_rate=0.000475, n_steps=1000, batch_size=1000, verbose=0, seed=0, tensorboard_log=log_path)
ppo_path = os.path.join('./Training/Saved_Models/PPO_car_best_Model_curriculum_obstacles1')

eval_callback = EvalCallback(eval_env=eval_env, best_model_save_path=ppo_path,
                              n_eval_episodes=10,
                             eval_freq=50000,verbose=1,
                             deterministic=True, render=False)
model.learn(total_timesteps=1000000,callback=eval_callback)
ppo_path = os.path.join('./Training/Saved_Models/PPO_Model_final_curriculum_obstacles1.zip')
model.save(ppo_path)

/Users/PrishitaRay1/opt/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/callbacks.py:399: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x15f549100> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x15f5717c0>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")
/Users/PrishitaRay1/opt/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Total number of obstacles in the track: 8
Total number of obstacles in the track: 9
Total number of obstacles in the track: 6
Total number of obstacles in the track: 10
Total number of obstacles in the track: 10
Total number of obstacles in the track: 9
Total number of obstacles in the track: 8
Total number of obstacles in the track: 9
Total number of obstacles in the track: 9
Total number of obstacles in the track: 8
Total number of obstacles in the track: 8
Eval num_timesteps=50000, episode_reward=-4.97 +/- 37.30
Episode length: 1000.00 +/- 0.00
New best mean reward!
Total number of obstacles in the track: 10
Total number of obstacles in the track: 9
Total number of obstacles in the track: 7
Total number of obstacles in the track: 10
Total number of obstacles in the track: 7
Total number of obstacles in the track: 6
Total number of obstacles in the track: 10
Total number of obstacles in the track: 7
Total number of obstacles in the track: 8
Total number of obstacles in the track: 9
T

Total number of obstacles in the track: 8
Total number of obstacles in the track: 7
Total number of obstacles in the track: 7
Total number of obstacles in the track: 7
Total number of obstacles in the track: 8
Eval num_timesteps=750000, episode_reward=682.80 +/- 130.24
Episode length: 964.80 +/- 54.54
New best mean reward!
Total number of obstacles in the track: 8
Total number of obstacles in the track: 7
Total number of obstacles in the track: 6
Total number of obstacles in the track: 9
Total number of obstacles in the track: 5
Total number of obstacles in the track: 10
Total number of obstacles in the track: 10
Total number of obstacles in the track: 6
Total number of obstacles in the track: 8
Total number of obstacles in the track: 8
Total number of obstacles in the track: 8
Eval num_timesteps=800000, episode_reward=419.57 +/- 275.64
Episode length: 1000.00 +/- 0.00
Total number of obstacles in the track: 5
Total number of obstacles in the track: 8
Total number of obstacles in the t

**4.Evaluating Models**- Best Model

In [4]:
ppo_path = os.path.join('./Training/Saved_Models/PPO_car_best_Model_curriculum_obstacles/best_model.zip')

In [5]:
best_model = PPO.load(ppo_path, env=eval_env)

In [6]:
def custom_evaluate_policy(
    model: "type_aliases.PolicyPredictor",
    env: Union[gym.Env, VecEnv],
    n_eval_episodes: int = 10,
    deterministic: bool = True,
    render: bool = False,
    callback: Optional[Callable[[Dict[str, Any], Dict[str, Any]], None]] = None,
    reward_threshold: Optional[float] = None,
    return_episode_rewards: bool = False,
    warn: bool = True,
) -> Union[Tuple[float, float], Tuple[List[float], List[int]]]:
    """
    Runs policy for ``n_eval_episodes`` episodes and returns average reward.
    If a vector env is passed in, this divides the episodes to evaluate onto the
    different elements of the vector env. This static division of work is done to
    remove bias. See https://github.com/DLR-RM/stable-baselines3/issues/402 for more
    details and discussion.

    .. note::
        If environment has not been wrapped with ``Monitor`` wrapper, reward and
        episode lengths are counted as it appears with ``env.step`` calls. If
        the environment contains wrappers that modify rewards or episode lengths
        (e.g. reward scaling, early episode reset), these will affect the evaluation
        results as well. You can avoid this by wrapping environment with ``Monitor``
        wrapper before anything else.

    :param model: The RL agent you want to evaluate. This can be any object
        that implements a `predict` method, such as an RL algorithm (``BaseAlgorithm``)
        or policy (``BasePolicy``).
    :param env: The gym environment or ``VecEnv`` environment.
    :param n_eval_episodes: Number of episode to evaluate the agent
    :param deterministic: Whether to use deterministic or stochastic actions
    :param render: Whether to render the environment or not
    :param callback: callback function to do additional checks,
        called after each step. Gets locals() and globals() passed as parameters.
    :param reward_threshold: Minimum expected reward per episode,
        this will raise an error if the performance is not met
    :param return_episode_rewards: If True, a list of rewards and episode lengths
        per episode will be returned instead of the mean.
    :param warn: If True (default), warns user about lack of a Monitor wrapper in the
        evaluation environment.
    :return: Mean reward per episode, std of reward per episode.
        Returns ([float], [int]) when ``return_episode_rewards`` is True, first
        list containing per-episode rewards and second containing per-episode lengths
        (in number of steps).
    """
    is_monitor_wrapped = False
    # Avoid circular import
    from stable_baselines3.common.monitor import Monitor

    if not isinstance(env, VecEnv):
        env = DummyVecEnv([lambda: env])  # type: ignore[list-item, return-value]

    is_monitor_wrapped = is_vecenv_wrapped(env, VecMonitor) or env.env_is_wrapped(Monitor)[0]

    if not is_monitor_wrapped and warn:
        warnings.warn(
            "Evaluation environment is not wrapped with a ``Monitor`` wrapper. "
            "This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. "
            "Consider wrapping environment first with ``Monitor`` wrapper.",
            UserWarning,
        )

    n_envs = env.num_envs
    episode_rewards = []
    episode_lengths = []
    episode_collisions = []
    episode_collision_ratios = []
    episode_tiles = []
    episode_grass_ratios = []

    episode_counts = np.zeros(n_envs, dtype="int")
    # Divides episodes among different sub environments in the vector as evenly as possible
    episode_count_targets = np.array([(n_eval_episodes + i) // n_envs for i in range(n_envs)], dtype="int")

    current_rewards = np.zeros(n_envs)
    current_lengths = np.zeros(n_envs, dtype="int")
    observations = env.reset()
    states = None
    episode_starts = np.ones((env.num_envs,), dtype=bool)
    while (episode_counts < episode_count_targets).any():
        actions, states = model.predict(
            observations,  # type: ignore[arg-type]
            state=states,
            episode_start=episode_starts,
            deterministic=deterministic,
        )
        new_observations, rewards, dones, infos = env.step(actions)
        current_rewards += rewards
        current_lengths += 1
        for i in range(n_envs):
            if episode_counts[i] < episode_count_targets[i]:
                # unpack values so that the callback can access the local variables
                reward = rewards[i]
                done = dones[i]
                info = infos[i]
                episode_starts[i] = done

                if callback is not None:
                    callback(locals(), globals())

                if dones[i]:
                    if is_monitor_wrapped:
                        # Atari wrapper can send a "done" signal when
                        # the agent loses a life, but it does not correspond
                        # to the true end of episode
                        if "episode" in info.keys():
                            # Do not trust "done" with episode endings.
                            # Monitor wrapper includes "episode" key in info if environment
                            # has been wrapped with it. Use those rewards instead.
                            episode_rewards.append(info["episode"]["r"])
                            episode_lengths.append(info["episode"]["l"])
                    
                            # Only increment at the real end of an episode
                            episode_counts[i] += 1
                    else:
                        episode_rewards.append(current_rewards[i])
                        episode_lengths.append(current_lengths[i])
                        
                        if "num_obstacles" in infos[i].keys():
                            episode_collisions.append(infos[i]["num_collisions"])
                            episode_collision_ratios.append(infos[i]["num_collisions"]/infos[i]["num_obstacles"])
                        
                        episode_tiles.append(infos[i]["tiles"])
                        episode_grass_ratios.append(infos[i]["grass_time"]/infos[i]["total_time"])
                        
                        episode_counts[i] += 1
                    current_rewards[i] = 0
                    current_lengths[i] = 0

        observations = new_observations

        if render:
            env.render()

    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)
    mean_collisions = np.mean(episode_collisions)
    mean_collision_ratios = np.mean(episode_collision_ratios)
    mean_tiles = np.mean(episode_tiles)
    mean_grass_ratios = np.mean(episode_grass_ratios)
    
    if reward_threshold is not None:
        assert mean_reward > reward_threshold, "Mean reward below threshold: " f"{mean_reward:.2f} < {reward_threshold:.2f}"
    if return_episode_rewards:
        return episode_rewards, episode_lengths
    return mean_reward, std_reward, mean_collisions, mean_collision_ratios, mean_tiles, mean_grass_ratios


In [7]:
evalue = custom_evaluate_policy(best_model, eval_env, n_eval_episodes=500, render = False)
eval_env.close()
evalue

/var/folders/5r/58zq17s164z4sxr5htkjbnch0000gp/T/ipykernel_23148/1992896862.py:57: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(663.8577228378505,
 133.57166080533986,
 0.932,
 0.12012632922632924,
 236.172,
 0.026208400941482225)